In [1]:
%config Completer.use_jedi = False

In [2]:
import os
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import datetime

In [3]:
from models.ResNet import resnet50

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input (InputLayer)              [(None, 32, 32, 3)]  0                                            
__________________________________________________________________________________________________
zero_padding2d (ZeroPadding2D)  (None, 38, 38, 3)    0           input[0][0]                      
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 16, 16, 64)   9408        zero_padding2d[0][0]             
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 16, 16, 64)   256         conv2d[0][0]                     
______________________________________________________________________________________________

In [4]:
from modules.TFR_load import TFR_load

In [5]:
# Hyper params

BATCH_SIZE     = 512
NUM_OF_CLASS   = 10
LEARNING_RATE  = 1e-5
COSINE_DECAY   = True
NUM_TRAIN_DATA = 60000
NUM_TEST_DATA  = 10000
EPOCH = 100

TR_STEPS_PER_EPOCH = NUM_TRAIN_DATA//BATCH_SIZE
TE_STEPS_PER_EPOCH = NUM_TEST_DATA//BATCH_SIZE

In [6]:
import tensorflow_datasets as tfds

ds_name = 'cifar10'
builder = tfds.builder(ds_name)

tr_ds, te_ds = builder.as_dataset(split = ['train', 'test'], shuffle_files = True)

tr_ds = tr_ds.batch(BATCH_SIZE)
print(len(tr_ds))
te_ds = te_ds.batch(BATCH_SIZE)
print(len(te_ds))

98
20


In [7]:
out_ch = [16, 32, 64, 128]
filter_size = [(5,5),(5,5),(3,3),(3,3),(3,3)]

In [8]:
model = resnet50

In [9]:
optimizer = tf.keras.optimizers.Adam(learning_rate=LEARNING_RATE, epsilon=1e-08)
loss = tf.keras.losses.CategoricalCrossentropy()
accuracy = tf.keras.metrics.CategoricalAccuracy()

In [19]:
predict = tf.cast([[0,0,0,0,1],
                   [0,1,0,0,0]], dtype = tf.float32)

predict

<tf.Tensor: shape=(2, 5), dtype=float32, numpy=
array([[0., 0., 0., 0., 1.],
       [0., 1., 0., 0., 0.]], dtype=float32)>

In [20]:
target = tf.one_hot([1,1], 5)

target

<tf.Tensor: shape=(2, 5), dtype=float32, numpy=
array([[0., 1., 0., 0., 0.],
       [0., 1., 0., 0., 0.]], dtype=float32)>

In [21]:
accuracy.reset_states()
accuracy.update_state(predict,target)
accuracy.result()

<tf.Tensor: shape=(), dtype=float32, numpy=0.5>

In [11]:
current_time = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
train_log_dir = 'logs/gradient_tape/' + current_time + '/train'
test_log_dir = 'logs/gradient_tape/' + current_time + '/test'
train_summary_writer = tf.summary.create_file_writer(train_log_dir)
test_summary_writer = tf.summary.create_file_writer(test_log_dir)

In [12]:
for EP in range(EPOCH):
    EP += 0
    for tr_example, te_example in zip(tr_ds, te_ds):
        images, labels = tr_example["image"], tr_example["label"]
        with tf.GradientTape() as tape:
            logits = model(images / 255)
            loss_value = loss(tf.one_hot(labels, 10), logits)
            accuracy.update_state(tf.one_hot(labels, 10), logits)
            grads = tape.gradient(loss_value, model.trainable_variables)
        optimizer.apply_gradients(zip(grads, model.trainable_variables))
        
        images, labels = te_example["image"], te_example["label"]
        logits = model(images / 255)
        loss_value = loss(tf.one_hot(labels, 10), logits)
        accuracy.update_state(tf.one_hot(labels, 10), logits)
        
    with train_summary_writer.as_default():
        tf.summary.scalar('loss', loss_value, step=EP)
        tf.summary.scalar('acc', accuracy.result(), step=EP)
        accuracy.reset_state()
        
        for w in model.weights:
            tf.summary.histogram(w.name, w, step=EP)
        
    with test_summary_writer.as_default():
        tf.summary.scalar('loss', loss_value, step=EP)
        tf.summary.scalar('acc', accuracy.result(), step=EP)
        accuracy.reset_state()
        

KeyboardInterrupt: 